# fact_messaging_accumulating

### Importación de librerías

In [25]:
import yaml
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta, time

### Conexión a base y bodega de datos

In [26]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)
    configFuente = config['fuente']
    configBodega = config['bodega']

urlFuente = f"{configFuente['driver']}://{configFuente['user']}:{configFuente['password']}@{configFuente['host']}:{configFuente['port']}/{configFuente['db']}"
urlBodega = f"{configBodega['driver']}://{configBodega['user']}:{configBodega['password']}@{configBodega['host']}:{configBodega['port']}/{configBodega['db']}"

src = create_engine(urlFuente)
etl = create_engine(urlBodega)

### Extracción y transformación de datos

In [ ]:
# Cargar los datos de la tabla 'mensajeria_estadosservicio' desde la fuente 'src'
mensajeria_estadosservicio = pd.read_sql_table('mensajeria_estadosservicio', src)

# Eliminar columnas innecesarias
mensajeria_estadosservicio.drop(columns=["foto", "observaciones", "es_prueba", "foto_binary"], inplace=True)

# Función para llenar las fechas y horas de los servicios
def asignar_fecha_y_hora(mensajeria, biblioteca, id_servicio, indice, fecha_clave, hora_clave):
    fecha_base_datos = mensajeria.iloc[indice, 1].strftime("%Y-%m-%d")
    hora_base_datos = mensajeria.iloc[indice, 2].strftime('%H:%M')
    
    biblioteca[id_servicio][fecha_clave] = fecha_base_datos
    biblioteca[id_servicio][hora_clave] = hora_base_datos

# Función para calcular la duración entre dos eventos
def calcular_duracion(biblioteca, id_servicio, fecha_inicial, hora_inicial, fecha_final, hora_final, duracion_clave):
    fecha_a = biblioteca[id_servicio][fecha_inicial]
    hora_a = biblioteca[id_servicio][hora_inicial]
    
    fecha_b = biblioteca[id_servicio][fecha_final]
    hora_b = biblioteca[id_servicio][hora_final]

    if fecha_a and hora_a and fecha_b and hora_b:
        datetime_a = datetime.strptime(f"{fecha_a} {hora_a}", "%Y-%m-%d %H:%M")
        datetime_b = datetime.strptime(f"{fecha_b} {hora_b}", "%Y-%m-%d %H:%M")
        
        duracion_minutos = (datetime_b - datetime_a).total_seconds() / 60
        biblioteca[id_servicio][duracion_clave] = duracion_minutos

# Crear la biblioteca que almacenará los datos de servicios
biblioteca_servicios = {}

# Llenar la biblioteca con la información de los servicios
for indice, fila in mensajeria_estadosservicio.iterrows():
    id_servicio = mensajeria_estadosservicio.iloc[indice, 4]
    id_estado = mensajeria_estadosservicio.iloc[indice, 3]

    if id_servicio not in biblioteca_servicios:
        biblioteca_servicios[id_servicio] = {
            'id_servicio': id_servicio,
            'fecha_inicio': None, 
            'hora_inicio': None,
            'fecha_asignacion': None,
            'hora_asignacion': None,
            'fecha_recogida': None,
            'hora_recogida': None,
            'fecha_entrega': None,
            'hora_entrega': None,
            'fecha_cierre': None,
            'hora_cierre': None,
            'duracion_inicio_asignacion': None,
            'duracion_asignacion_recogida': None,
            'duracion_recogida_entrega': None,
            'duracion_entrega_cierre': None
        }
    
    if id_estado == 1:
        asignar_fecha_y_hora(mensajeria_estadosservicio, biblioteca_servicios, id_servicio, indice, 'fecha_inicio', 'hora_inicio')
    elif id_estado == 2:
        asignar_fecha_y_hora(mensajeria_estadosservicio, biblioteca_servicios, id_servicio, indice, 'fecha_asignacion', 'hora_asignacion')
    elif id_estado == 4:
        asignar_fecha_y_hora(mensajeria_estadosservicio, biblioteca_servicios, id_servicio, indice, 'fecha_recogida', 'hora_recogida')
    elif id_estado == 5:
        asignar_fecha_y_hora(mensajeria_estadosservicio, biblioteca_servicios, id_servicio, indice, 'fecha_entrega', 'hora_entrega')
    elif id_estado == 6:
        asignar_fecha_y_hora(mensajeria_estadosservicio, biblioteca_servicios, id_servicio, indice, 'fecha_cierre', 'hora_cierre')

# Calcular las duraciones entre los diferentes estados del servicio
for id_servicio in biblioteca_servicios:
    calcular_duracion(biblioteca_servicios, id_servicio, 'fecha_inicio', 'hora_inicio', 'fecha_asignacion', 'hora_asignacion', 'duracion_inicio_asignacion')
    calcular_duracion(biblioteca_servicios, id_servicio, 'fecha_asignacion', 'hora_asignacion', 'fecha_recogida', 'hora_recogida', 'duracion_asignacion_recogida')
    calcular_duracion(biblioteca_servicios, id_servicio, 'fecha_recogida', 'hora_recogida', 'fecha_entrega', 'hora_entrega', 'duracion_recogida_entrega')
    calcular_duracion(biblioteca_servicios, id_servicio, 'fecha_entrega', 'hora_entrega', 'fecha_cierre', 'hora_cierre', 'duracion_entrega_cierre')

# Crear un DataFrame a partir de la biblioteca
fact_mensajeria = pd.DataFrame.from_dict(biblioteca_servicios, orient='index')

# Convertir fechas y horas a sus respectivos formatos
fact_mensajeria['fecha_inicio'] = pd.to_datetime(fact_mensajeria['fecha_inicio'], errors='coerce')
fact_mensajeria['fecha_asignacion'] = pd.to_datetime(fact_mensajeria['fecha_asignacion'], errors='coerce')
fact_mensajeria['fecha_recogida'] = pd.to_datetime(fact_mensajeria['fecha_recogida'], errors='coerce')
fact_mensajeria['fecha_entrega'] = pd.to_datetime(fact_mensajeria['fecha_entrega'], errors='coerce')
fact_mensajeria['fecha_cierre'] = pd.to_datetime(fact_mensajeria['fecha_cierre'], errors='coerce')

fact_mensajeria['hora_inicio'] = pd.to_datetime(fact_mensajeria['hora_inicio'], format='%H:%M', errors='coerce').dt.time
fact_mensajeria['hora_asignacion'] = pd.to_datetime(fact_mensajeria['hora_asignacion'], format='%H:%M', errors='coerce').dt.time
fact_mensajeria['hora_recogida'] = pd.to_datetime(fact_mensajeria['hora_recogida'], format='%H:%M', errors='coerce').dt.time
fact_mensajeria['hora_entrega'] = pd.to_datetime(fact_mensajeria['hora_entrega'], format='%H:%M', errors='coerce').dt.time
fact_mensajeria['hora_cierre'] = pd.to_datetime(fact_mensajeria['hora_cierre'], format='%H:%M', errors='coerce').dt.time


In [ ]:
# Rellenar los valores faltantes en las duraciones utilizando la media
fact_mensajeria['duracion_inicio_asignacion'].fillna(round(fact_mensajeria['duracion_inicio_asignacion'].mean(), 0), inplace=True)
fact_mensajeria['duracion_asignacion_recogida'].fillna(round(fact_mensajeria['duracion_asignacion_recogida'].mean(), 0), inplace=True)
fact_mensajeria['duracion_recogida_entrega'].fillna(round(fact_mensajeria['duracion_recogida_entrega'].mean(), 0), inplace=True)
fact_mensajeria['duracion_entrega_cierre'].fillna(round(fact_mensajeria['duracion_entrega_cierre'].mean(), 0), inplace=True)

# Función para ajustar fechas y horas con base en la duración
def llenar_fecha_y_hora2(fact_mensajeria, indice, duracion_base, fecha_modificar, fecha_base, hora_modificar, hora_base):
    duracion = fact_mensajeria.loc[indice, duracion_base]

    dias_d = duracion // 1440
    minutos_d = duracion % 1440

    if dias_d == 0:
        fact_mensajeria.loc[indice, fecha_modificar] = fact_mensajeria.loc[indice, fecha_base]

        hora_base_valor = fact_mensajeria.loc[indice, hora_base]
        hora_modificada = timedelta(hours=hora_base_valor.hour, minutes=hora_base_valor.minute) + timedelta(minutes=minutos_d)

        if isinstance(hora_modificada, timedelta):
            total_segundos = int(hora_modificada.total_seconds())
            horas = (total_segundos // 3600) % 24
            minutos = (total_segundos % 3600) // 60
            segundos = total_segundos % 60
            
            hora_modificada = time(hour=horas, minute=minutos, second=segundos)

        fact_mensajeria.loc[indice, hora_modificar] = hora_modificada

# Iterar sobre las filas de fact_mensajeria para ajustar las fechas y horas
for indice, fila in fact_mensajeria.iterrows():
    for columna, valor in fila.items():
        if pd.isna(valor):
            if columna == "fecha_inicio":
                duracion = fact_mensajeria.loc[indice, 'duracion_inicio_asignacion']

                dias_d = duracion // 1440
                minutos_d = duracion % 1440

                if dias_d == 0:
                    fact_mensajeria.loc[indice, 'fecha_inicio'] = fact_mensajeria.loc[indice, 'fecha_asignacion']
                    hora_asignacion = fact_mensajeria.loc[indice, 'hora_asignacion']
                    hora_inicio = (timedelta(hours=hora_asignacion.hour, minutes=hora_asignacion.minute) - timedelta(minutes=minutos_d))

                    if isinstance(hora_inicio, timedelta):
                        total_segundos = int(hora_inicio.total_seconds())
                        horas = (total_segundos // 3600) % 24
                        minutos = (total_segundos % 3600) // 60
                        segundos = total_segundos % 60
                        
                        hora_inicio = time(hour=horas, minute=minutos, second=segundos)

                    fact_mensajeria.loc[indice, 'hora_inicio'] = hora_inicio

            if columna == "fecha_asignacion":
                llenar_fecha_y_hora2(fact_mensajeria, indice, 'duracion_inicio_asignacion', 'fecha_asignacion', 'fecha_inicio', 'hora_asignacion', 'hora_inicio')

            if columna == "hora_asignacion":
                duracion = fact_mensajeria.loc[indice, 'duracion_inicio_asignacion']
                minutos_d = duracion % 1440

                hora_inicio = fact_mensajeria.loc[indice, 'hora_inicio']
                hora_asignacion = (timedelta(hours=hora_inicio.hour, minutes=hora_inicio.minute) + timedelta(minutes=minutos_d))

                if isinstance(hora_asignacion, timedelta):
                    total_segundos = int(hora_asignacion.total_seconds())
                    horas = (total_segundos // 3600) % 24
                    minutos = (total_segundos % 3600) // 60
                    segundos = total_segundos % 60
                    
                    hora_asignacion = time(hour=horas, minute=minutos, second=segundos)

                fact_mensajeria.loc[indice, 'hora_asignacion'] = hora_asignacion

            if columna == "fecha_recogida":
                llenar_fecha_y_hora2(fact_mensajeria, indice, 'duracion_asignacion_recogida', 'fecha_recogida', 'fecha_asignacion', 'hora_recogida', 'hora_asignacion')

            if columna == "fecha_entrega":
                llenar_fecha_y_hora2(fact_mensajeria, indice, 'duracion_recogida_entrega', 'fecha_entrega', 'fecha_recogida', 'hora_entrega', 'hora_recogida')

            if columna == "fecha_cierre":
                llenar_fecha_y_hora2(fact_mensajeria, indice, 'duracion_entrega_cierre', 'fecha_cierre', 'fecha_entrega', 'hora_cierre', 'hora_entrega')

# Cargar las tablas 'dim_fecha' y 'dim_hora'
dim_fecha = pd.read_sql_table('dim_fecha', etl)
dim_hora = pd.read_sql_table('dim_hora', etl)

# Unir la tabla fact_mensajeria con la dimensión fecha y reemplazar las claves
for col_fecha in ['fecha_inicio', 'fecha_asignacion', 'fecha_recogida', 'fecha_entrega', 'fecha_cierre']:
    fact_mensajeria = fact_mensajeria.merge(
        dim_fecha[['fecha', 'key_dim_fecha']], 
        left_on=col_fecha, 
        right_on='fecha', 
        how='left'
    ).drop(columns=[col_fecha]).rename(columns={'key_dim_fecha': col_fecha}).drop(columns=['fecha'])
    fact_mensajeria[col_fecha] = pd.to_numeric(fact_mensajeria[col_fecha], errors='coerce').astype('Int64')

# Unir la tabla fact_mensajeria con la dimensión hora y reemplazar las claves
for col_hora in ['hora_inicio', 'hora_asignacion', 'hora_recogida', 'hora_entrega', 'hora_cierre']:
    fact_mensajeria = fact_mensajeria.merge(
        dim_hora[['hora', 'key_dim_hora']], 
        left_on=col_hora, 
        right_on='hora', 
        how='left'
    ).drop(columns=[col_hora]).rename(columns={'key_dim_hora': col_hora}).drop(columns=['hora'])
    fact_mensajeria[col_hora] = pd.to_numeric(fact_mensajeria[col_hora], errors='coerce').astype('Int64')

# Añadir una columna de clave primaria acumulada
fact_mensajeria["clave_fact_mensajeria_acumulada"] = range(1, len(fact_mensajeria) + 1)

# Mostrar el DataFrame final
fact_mensajeria


C:\Users\jpant\AppData\Local\Temp\ipykernel_12112\1965599818.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  fact_messaging['start_to_assignment_duration'].fillna(round(fact_messaging['start_to_assignment_duration'].mean(), 0), inplace=True)
C:\Users\jpant\AppData\Local\Temp\ipykernel_12112\1965599818.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the i

,key_service,start_to_assignment_duration,assignment_to_pick_up_duration,pick_up_to_delivery_duration,delivery_to_closing_duration,key_start_date,key_assignment_date,key_pick_up_date,key_delivery_date,key_closing_date,key_start_time,key_assignment_time,key_pick_up_time,key_delivery_time,key_closing_time,key_fact_messaging_accumulating
0,226,30.0,46.0,26.0,292.0,393,394,394,394,394,1438,28,74,100,392,1
1,79,4477.0,202.0,38825.0,1.0,365,368,368,395,395,821,978,1180,1125,1126,2
2,613,7.0,18.0,34.0,12.0,402,402,402,402,402,636,643,661,695,707,3
3,376,1078.0,0.0,0.0,292.0,396,397,397,397,397,1253,891,891,891,1183,4
4,7164,69.0,6.0,81.0,292.0,462,462,462,462,462,962,1031,1037,1118,1410,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28425,28464,37.0,13.0,19.0,292.0,609,609,609,609,609,769,806,819,838,1130,28426
28426,28465,10.0,52.0,49.0,292.0,609,609,609,609,609,812,822,874,923,1215,28427
28427,28466,60.0,1.0,104.0,292.0,609,609,609,609,609,844,904,905,1009,1301,28428
28428,28467,49.0,90.0,104.0,292.0,609,609,609,609,609,853,902,992,1096,1388,28429


### Carga de datos

In [ ]:
fact_mensajeria.to_sql("hecho_entrega_mensajeria_acumulada", etl, index=False, if_exists="replace")

430